In [52]:
import requests
import json
import pandas as pd
from datetime import datetime


### Загружаем сохранённые данные о сигналами 

In [53]:
alert_df = pd.read_csv("alert_from_algo1_091123.csv")

In [54]:
alert_df

,Unnamed: 0,symbol,emoji,kline,alert_price,Time_(MSK)
0,0,ANTUSDT,💥💥💥,1 kline,alert price: 5.201,2023-11-09 18:51:52.578000
1,1,ANTUSDT,💥💥💥,1 kline,alert price: 5.201,2023-11-09 18:51:52.578000
2,2,PENDLEUSDT,💥💥💥,1 kline,alert price: 1.0423,2023-11-09 18:50:35.789000
3,3,PENDLEUSDT,💥💥💥,1 kline,alert price: 1.0423,2023-11-09 18:50:35.789000
4,4,YFIUSDT,💥💥💥,1 kline,alert price: 6868.0,2023-11-09 18:48:24.505000
...,...,...,...,...,...,...
13989,14352,ANTUSDT,🌊🌊🌊,2 kline,alert price: 4.133,2023-09-14 19:32:27.747000
13990,14353,BCHUSDT,🌊🌊🌊,2 kline,alert price: 208.66,2023-09-14 19:31:45.800000
13991,14354,AXSUSDT,💥💥💥,1 kline,alert price: 4.654,2023-09-14 19:31:29.016000
13992,14355,SPELLUSDT,💥💥💥,1 kline,alert price: 0.0004563,2023-09-14 19:21:40.284000


### Получение данных с Binance

In [55]:
def get_historical_klines(symbol, interval, start_time, end_time):
    """
    Функция для получения исторических данных свечей (клайнов) с Binance по заданному символу (паре торгов),
    интервалу времени, начальному и конечному времени.
    Возвращает данные в виде датафрейма pandas.
    """
    url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&startTime={start_time}&endTime={end_time}"
    response = requests.get(url)
    data = json.loads(response.text)
    df = create_dataframe(data)
    return df

### Подготавливаем шаблон будущего датафрейма

In [56]:
def create_dataframe(data):
    """
    Функция для создания датафрейма pandas на основе полученных данных с Binance.
    Преобразует числовые столбцы в числовой формат и приводит столбцы с временем к формату даты и времени.
    Возвращает подготовленный датафрейм.
    """
    df = pd.DataFrame(data, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
                                     'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
                                     'taker_buy_quote_asset_volume', 'ignore'])
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    df = convert_numeric_columns(df)
    return df


In [57]:
def convert_numeric_columns(df):
    """
    Функция для преобразования числовых столбцов датафрейма в числовой формат.
    Возвращает датафрейм с преобразованными столбцами.
    """
    numeric_columns = ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume',
                       'taker_buy_quote_asset_volume']
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)
    return df


### Вычисление нужного временного диапазона (диапазона свечей)

In [58]:
def calculate_time_range(alert_time):
    """
    Функция для вычисления начального и конечного времени для получения исторических данных свечей.
    Начальное время задается как 160 минут (2 часа 40 минут) до времени алерта, а конечное время - 120 минут (2 часа) после времени алерта.
    Возвращает начальное и конечное время в миллисекундах.
    """
    start_time = int((alert_time - pd.Timedelta(minutes=160)).timestamp() * 1000)
    end_time = int((alert_time + pd.Timedelta(minutes=120)).timestamp() * 1000)
    return start_time, end_time


### Вычисление длины споротивления

In [59]:
def calculate_resistance_length(data, alert_price):
    """
    Функция для вычисления длины сопротивления на основе данных свечей и цены алерта.
    Проходит по каждой свече в данных и увеличивает счетчик resistance_length,
    пока цена не опускается ниже или равна цене алерта.
    Возвращает длину сопротивления.
    """
    resistance_length = 0
    for _, row in data.iterrows():
        if row['high'] >= alert_price:
            resistance_length += 1
        else:
            break
    return resistance_length


### Cобираем нужные метрики (макс цена, длина сопротивления, >3%)

In [60]:
def calculate_metrics(data, alert_price):
    """
    Функция для вычисления необходимых метрик на основе данных свечей и цены алерта.
    Вычисляет максимальную цену, процент изменения цены относительно цены алерта,
    длину сопротивления и определяет, является ли сигнал эффективным (если процент изменения >= 3%).
    Возвращает максимальную цену, процент изменения, длину сопротивления и флаг эффективности.
    """
    max_price = data['high'].max()
    price_change_percentage = ((max_price - alert_price) / alert_price) * 100
    resistance_length = calculate_resistance_length(data, alert_price)
    is_effective = price_change_percentage >= 3
    return max_price, price_change_percentage, resistance_length, is_effective


### Cоздаём функцию, которая будет давать нам нужный диапазон свечей

In [61]:
def process_alerts(alert_df):
    """
    Функция для обработки сигналов алертов.
    Проходит по каждому сигналу в датафрейме алертов и выполняет следующие действия:
    - Извлекает символ, время алерта и цену алерта
    - Вычисляет нужный временной диапазон (диапазон свечей) на основе времени алерта
    - Получает исторические данные свечей с Binance
    - Вычисляет необходимые метрики (максимальная цена, длина сопротивления, >3%)
    - Добавляет результаты в список processed_data
    Возвращает датафрейм с обработанными данными.
    """
    processed_data = []
    for index, row in alert_df.iterrows():
        symbol = row['symbol']
        alert_time = pd.to_datetime(row['Time_(MSK)'])
        alert_price = float(row['alert_price'].split(': ')[1])
        start_time, end_time = calculate_time_range(alert_time)
        data = get_historical_klines(symbol, '1m', start_time, end_time)
        max_price, price_change_percentage, resistance_length, is_effective = calculate_metrics(data, alert_price)
        processed_data.append({
            'symbol': symbol,
            'alert_time': alert_time,
            'alert_price': alert_price,
            'max_price': max_price,
            'price_change_percentage': price_change_percentage,
            'resistance_length': resistance_length,
            'is_effective': is_effective
        })
    return pd.DataFrame(processed_data)

In [ ]:
processed_data = process_alerts(alert_df)


In [ ]:
processed_data.to_csv("Lebiadzevich_processed_data.csv")